In [17]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Alphabet import IUPAC
import pandas as pd
import re

for record in SeqIO.parse("../syn3_9_01.fa", "fasta"):
    ori_seq = Seq(str(record.seq), IUPAC.unambiguous_dna)
    rev_com_seq = str(ori_seq.reverse_complement())
total_len = len(ori_seq)  

In [33]:
foundlist = []
dic = pd.read_csv("../PCRtag_yeast_chr03_3_40.csv", header=0)
for idx, row in dic.iterrows():
    ori_result = ori_seq.find(row["Forward Syn"])
    if ori_result != -1:
        WT = str(ori_seq[int(ori_result-25):ori_result]+row['Forward WT']+ori_seq[int(ori_result+len(row["Forward Syn"])):ori_result+len(row["Forward Syn"])+26])
        foundlist.append([row["Amplicon"], int(ori_result-25), int(ori_result+len(row["Forward Syn"])+26), "Forward", WT])
    else:
        print(idx, row["Forward Syn"], " not found!")
    com_result = rev_com_seq.find(row["Reverse Syn"])
    if com_result != -1:
        WT = str(rev_com_seq[int(com_result-25):com_result]+row['Reverse WT']+rev_com_seq[int(com_result+len(row["Reverse Syn"])):com_result+len(row["Reverse Syn"])+26])
        foundlist.append([row["Amplicon"], int(total_len-int(com_result+len(row["Reverse Syn"])+26)), int(total_len-int(com_result-25)), "Reverse", WT])
    else:
        print(idx, row["Reverse Syn"], " not found!")

frames = pd.DataFrame(foundlist, columns=['Amplicon', 'syn_FWD_start', 'syn_FWD_end', 'strand', 'WT_5_to_3'])
frames.to_csv('../test.csv', index=False)

67 TAGTTCCAAAGGGCTAGCGGTTCGGCTA  not found!
67 CGCTGGTATCTTTGTCTCTACCAGCTCT  not found!


In [28]:
output = []
for idx, row in frames.iterrows():
    WT = row['WT_5_to_3']
    if row['strand'] == "Forward":
        seq = ori_seq
        start = row["syn_FWD_start"]
        end = row["syn_FWD_end"]
    elif row['strand'] == "Reverse":
        seq = rev_com_seq
        start = total_len - row["syn_FWD_end"]
        end = total_len - row["syn_FWD_start"]
    frame = str(seq[start:end])
    target = [n.start() for n in re.finditer(".GG", frame)]
    if len(target) != 0: # make sure NGG exists on syn
        for f in target:
            f_abs = f + start
            if row['strand'] == "Forward":
                newstart = f_abs-20 
                newend = f_abs+3
            elif row['strand'] == "Reverse":
                newstart = total_len - (f_abs+3)
                newend = total_len - (f_abs-20)
            GGpos = [n.start() for n in re.finditer("GG", WT)] # find GG on WT
            if len(GGpos) != 0: # GG found on WT
                ce = 0
                ct = 0
                for GG in GGpos:
                    eight = WT[GG-9:GG-1] # get -8 to -1
                    if eight == str(seq[f_abs-8:f_abs]): # -8 to -1 same
                        twelve = WT[GG-21:GG-9] # get -20 to -9
                        if twelve == str(seq[f_abs-20:f_abs-8]): # -20 to -9 same
                            break
                        else: # -20 to -9 different for this GG
                            ct+=1
                    else: # -8 to -1 different for this GG
                        ce+=1
                        ct+=1
                if ce == len(GGpos) and ct == len(GGpos):  # -8 to -1 different for all GG
                    output.append([str(seq[f_abs-20:f_abs+3]), "8 different", row['strand'], row['Amplicon'], newstart, newend])
                    print("eight different: " + str(seq[f_abs-20:f_abs+3]))
                elif ct == len(GGpos): # -20 to -9 different for all GG
                    output.append([str(seq[f_abs-20:f_abs+3]), "12 different", row['strand'], row['Amplicon'], newstart, newend]) # use with causion
                    print("twelve caution: " + str(seq[f_abs-20:f_abs+3]))
                else:
                    print("All same found! Do not use!") # do not use   
            else: # GG not found on WT
                output.append([str(seq[f_abs-20:f_abs+3]), "GG different", row['strand'], row['Amplicon'], newstart, newend])
                print("GG not found in WT: " + str(seq[f_abs-20:f_abs+3]))           

                
                
                  

eight different: CGCTTCTTCCAAATCTTTGATGG
eight different: AACAAGTATTGCTACCAGCTTGG
eight different: GCAGCCAAATCACATGATACTGG
All same found! Do not use!
eight different: GTTCCAAACGTCGGGTTGCTAGG
eight different: TTAGCATAGCTAGATAGCAAAGG
twelve caution: TCGTGGTCCTGACACCCAAATGG
twelve caution: GAAGACAAAACCCAAGCAATTGG
twelve caution: CCAAGCAATTGGTATTCCACAGG
eight different: TCGTCATTCTGTACTGGTTTTGG
eight different: CTGAAAACACCTCTCACGACAGG
twelve caution: AAACACCTCTCACGACAGGACGG
twelve caution: CCTCTCACGACAGGACGGCCAGG
eight different: ACCCAACGAATCGACTCTTCTGG
twelve caution: CGTTAAACACCTTGTCTTGTTGG
twelve caution: TTGTCTTGTTGGATGCTGACAGG
eight different: GGATGCTGACAGGTATTCGTTGG
eight different: TGACAGGTATTCGTTGGGTTTGG
eight different: GATGAGAACGATAATACTAATGG
twelve caution: AACGATAATACTAATGGAAATGG
eight different: GATAACCCACCTGAACTAACCGG
eight different: CCACCTGAACTAACCGGTAATGG
eight different: ATTTCCAAGTCGACTTTTCGTGG
eight different: ACTTTTCGTGGTAGTTGAGCTGG
eight different: TTTCGTGGTAGTTGAGCTGG

In [29]:
summary = pd.DataFrame(output, columns=['5_to_3_sequence', 'pattern', 'strand', 'amplicon', 'syn_FWD_start', 'syn_FWD_end'])
summary

,5_to_3_sequence,pattern,strand,amplicon,syn_FWD_start,syn_FWD_end
0,CGCTTCTTCCAAATCTTTGATGG,8 different,Forward,YCL064C_amp1,6336,6359
1,AACAAGTATTGCTACCAGCTTGG,8 different,Reverse,YCL064C_amp1,6611,6634
2,GCAGCCAAATCACATGATACTGG,8 different,Forward,YCL063W_amp1,8302,8325
3,GTTCCAAACGTCGGGTTGCTAGG,8 different,Reverse,YCL063W_amp1,8617,8640
4,TTAGCATAGCTAGATAGCAAAGG,8 different,Forward,YCL061C_amp1,11445,11468
5,TCGTGGTCCTGACACCCAAATGG,12 different,Reverse,YCL061C_amp1,11884,11907
6,GAAGACAAAACCCAAGCAATTGG,12 different,Reverse,YCL061C_amp1,11835,11858
7,CCAAGCAATTGGTATTCCACAGG,12 different,Reverse,YCL061C_amp1,11824,11847
8,TCGTCATTCTGTACTGGTTTTGG,8 different,Forward,YCL061C_amp2,11538,11561
9,CTGAAAACACCTCTCACGACAGG,8 different,Reverse,YCL061C_amp2,12096,12119


CCAAAAATAATACCAACCAAAGC
